# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("homepage_actions.csv")
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [64]:
sum(df['id'].duplicated())

1860

In [72]:
test = df['id'].duplicated(keep=False)
df[test]

,timestamp,id,group,action
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
11,2016-09-24 21:05:15.348935,601714,experiment,view
12,2016-09-24 21:06:27.553057,601714,experiment,click
14,2016-09-24 21:29:19.766467,487634,experiment,view
...,...,...,...,...
8167,2017-01-18 00:56:24.554729,344770,experiment,click
8179,2017-01-18 08:53:50.910310,615849,experiment,view
8180,2017-01-18 08:54:56.879682,615849,experiment,click
8181,2017-01-18 09:07:37.661143,795585,control,view


In [73]:
vids = set(df.loc[df["action"]=='view','id'].unique())
cids = set(df.loc[df["action"] =="click",'id'].unique())

print(f"Number of viewers = {len(vids)}")
print(f"Number of clickers = {len(cids)}")
print(f"Number of viewers with clickers removed = {len(vids - cids)}")
print(f"Number of clickers with viewers removed = {len(cids - vids)}")

#All who clicked also viewed

Number of viewers = 6328
Number of clickers = 1860
Number of viewers who didn't click = 4468
Number of clickers who didn't view = 0


In [77]:
ctids = set(df.loc[df["group"]=='control','id'].unique())
exids = set(df.loc[df["group"]=='experiment','id'].unique())

print(f"Number of controls = {len(ctids)}")
print(f"Number of experiments = {len(exids)}")
print(f"Number of controls with experimental removed = {len(ctids - exids)}")
print(f"Number of experiments with controls removed = {len(exids - ctids)}")

#No overlap between experimental and controls

Number of controls = 3332
Number of experiments = 2996
Number of controls not overlapping with experiments = 3332
Number of experiments not overlapping with controls = 2996


In [3]:
temp = df["timestamp"].str.split(" ", expand = True)
temp[0].agg([min,max])


min    2016-09-24
max    2017-01-18
Name: 0, dtype: object

In [5]:
temp[0].value_counts().sort_index()

2016-09-24    20
2016-09-25    78
2016-09-26    82
2016-09-27    71
2016-09-28    73
              ..
2017-01-14    63
2017-01-15    76
2017-01-16    81
2017-01-17    72
2017-01-18    23
Name: 0, Length: 117, dtype: int64

In [123]:
temp = df["timestamp"].str.split(" ", expand = True)
order = temp[0].unique()
temp[0].value_counts()

2016-12-10    100
2017-01-06     92
2016-11-22     91
2016-10-27     87
2016-11-23     87
             ... 
2016-11-12     53
2016-10-11     52
2016-11-13     40
2017-01-18     23
2016-09-24     20
Name: 0, Length: 117, dtype: int64

In [6]:
df.groupby('group')['action'].value_counts()

group       action
control     view      3332
            click      932
experiment  view      2996
            click      928
Name: action, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [ ]:
"""HO: The proportion of clicks received on the website by the experimental group is less than or equal to the control groups
Ha: The proportion of clicks received on the website by the experimental group is greater than the control groups"""

In [98]:
#Your code here
#We need to make the dataframe wide from long format so we need some sort of 'value' measure - 
# input a 1 for each row
df['one'] = 1
df.head()

,timestamp,id,group,action,inx,one
0,2016-09-24 17:42:27.839496,804196,experiment,view,1,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1,1
3,2016-09-24 19:59:02.646620,671993,control,view,1,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1,1


In [104]:
df_control = df[df["group"]=="control"].pivot(index = "id", columns = "action", values = "one")
df_control["click"].fillna(0, inplace = True)
df_control.head()

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [100]:
df_exp = df[df["group"]=="experiment"].pivot(index = "id", columns = "action", values = "one")
df_exp["click"].fillna(0, inplace = True)
df_exp.head()

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0


In [125]:
print(f"Number of Controls who clicked = {df_control['click'].sum()}")
print(f"Number of Experiments who clicked = {df_exp['click'].sum()}\n")

print(f"Sample Size Controls = {len(df_control)}")
print(f"Sample Size Experiments = {len(df_exp)}\n")

p_control_click = df_control['click'].sum() / len(df_control)
p_exp_click = df_exp['click'].sum() / len(df_exp)

print(f"Proportion of controls who clicked = {p_control_click}")
print(f"Proportion of experiments who clicked = {p_exp_click}")

Number of Controls who clicked = 932.0
Number of Experiments who clicked = 928.0

Sample Size Controls = 3332
Sample Size Experiments = 2996.0

Proportion of controls who clicked = 0.2797118847539016
Proportion of experiments who clicked = 0.3097463284379172


In [108]:
import scipy.stats as stats
stats.ttest_ind(df_control["click"], df_exp["click"], equal_var = False, alternative = "less")

Ttest_indResult(statistic=-2.615440020788211, pvalue=0.004466402814337101)

In [ ]:
"""The result is statistical significant at the 5% signficance level so we reject the null
hypothesis"""

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [109]:
#Your code here
2996.0 * 0.2797118847539016

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [126]:
#Your code here
n = 2996.0
pact = 928
pexp = 838.0
p = p_control_click
std = np.sqrt(n * p * (1-p))

z_score = pact - pexp / std
print(diff / std)

3.6632201602088235


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [114]:
#Your code here
1 - stats.norm.cdf(3.66322)

0.00012453222356945126

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Under both the non-parametric and parametric approaches we do wind up rejecting the null
hypothesis so in that regard we would be led to the same conclusion that the experimental website
does have a different click rate.  However, the difference in p-values of an order of magnitude
is notable and would lead me to believe the non-parametric approach is more conservative which
may be the better way to go.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.